In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pandas as pd
import numpy as np
import re

In [2]:
# Load data
df = pd.read_csv('./data/product_df.csv')
df = df[['Star Rating', 'Comment']]

In [3]:
def label_sentiment(rating):
    if rating in [1, 2]:
        return 'tiêu cực'
    elif rating == 3:
        return 'trung tính'
    elif rating in [4, 5]:
        return 'tích cực'
    else:
        return 'không rõ'  # Nếu có xếp hạng nằm ngoài khoảng 1-5

# Gắn nhãn cảm xúc cho mỗi đánh giá
df['Sentiment'] = df['Star Rating'].apply(label_sentiment)

# Hiển thị 5 hàng đầu tiên của dataframe với cột sentiment mới
print(df.head())

   Star Rating                                            Comment   Sentiment
0            4                      Điện thoại này dùng rất thích    tích cực
1            4                               sử dụng thấy cũng ok    tích cực
2            2                      Bảo hành ít quá, chỉ 12 tháng    tiêu cực
3            5                             Sản phẩm mượt, chạy êm    tích cực
4            3  Cho mình hỏi muốn khởi động lại máy hay tắt ng...  trung tính


In [4]:
# Kiểm tra và loại bỏ giá trị khuyết thiếu
print(df.isnull().sum())
df = df.dropna(subset=['Comment'])

Star Rating    0
Comment        0
Sentiment      0
dtype: int64


In [5]:
# Kiểm tra và loại bỏ dữ liệu trùng lặp
duplicate_comments = df[df.duplicated(['Comment'])]
print("Các dòng dữ liệu trùng lặp trong cột 'Comment':")
print(duplicate_comments)
df = df.drop_duplicates(['Comment'])
print("Shape after dropping duplicates:", df.shape)

Các dòng dữ liệu trùng lặp trong cột 'Comment':
      Star Rating                                   Comment Sentiment
48              4                                   Rất tốt  tích cực
76              5                                    Rất ok  tích cực
107             2                             pin tụt nhanh  tiêu cực
135             5                              sản phẩm tốt  tích cực
173             2                             hao pin nhanh  tiêu cực
177             5                              sản phẩm tốt  tích cực
181             5                                    Rất ok  tích cực
183             5                              sản phẩm tốt  tích cực
192             5                               sản phẩm ok  tích cực
205             5                               máy dùng ok  tích cực
225             4                                       Tốt  tích cực
227             5                               sản phẩm ok  tích cực
237             5                         

In [6]:
# Chuẩn hóa và làm sạch văn bản
def remove_special_characters(text):
    # Loại bỏ các ký tự đặc biệt, giữ lại chữ cái, số, và các dấu câu
    return re.sub(r'[^a-zA-ZÀ-ỹà-ỹ0-9\s]', '', text)

def to_lowercase(text):
    # Chuyển đổi văn bản về chữ thường
    return text.lower()

def normalize_text(text):
    text = remove_special_characters(text)
    text = to_lowercase(text)
    return text

df['Comment'] = df['Comment'].apply(normalize_text)
print(df.head())

   Star Rating                                            Comment   Sentiment
0            4                      điện thoại này dùng rất thích    tích cực
1            4                               sử dụng thấy cũng ok    tích cực
2            2                       bảo hành ít quá chỉ 12 tháng    tiêu cực
3            5                              sản phẩm mượt chạy êm    tích cực
4            3  cho mình hỏi muốn khởi động lại máy hay tắt ng...  trung tính


In [7]:
from underthesea import word_tokenize

# Function to tokenize and build vocab using underthesea
def tokenize_and_build_vocab_vietnamese(comment):
    tokens = word_tokenize(comment, format="text")
    return tokens.split()

# Tokenize comments and build vocabulary
tokenized_comments = df['Comment'].apply(tokenize_and_build_vocab_vietnamese)
all_tokens = [token for sublist in tokenized_comments for token in sublist]
vocab = FreqDist(all_tokens)

# Build vocab_list from vocab
vocab_list = list(vocab.keys())

print("Tổng số từ duy nhất:", len(vocab_list))
print("Các từ và tần suất xuất hiện:")
for word, frequency in vocab.most_common(10):  # Print only top 10 words
    print(f"{word}: {frequency}")


Tổng số từ duy nhất: 2386
Các từ và tần suất xuất hiện:
máy: 715
không: 659
dùng: 593
mua: 472
được: 422
rất: 409
pin: 347
tốt: 338
mình: 322
thì: 319


Mã hóa dữ liệu

In [8]:
from gensim.models import Word2Vec
import os
import pandas as pd

In [9]:
def read_data_with_labels(path):
    # Hàm này có thể được sửa đổi để phù hợp với định dạng dữ liệu cụ thể của bạn
    # Đọc dữ liệu và nhãn từ tập tin ở đường dẫn path
    reviews = []  # Danh sách các đánh giá
    labels = []   # Danh sách các nhãn tương ứng

    # Code đọc dữ liệu từ file ở đây và lưu vào reviews và labels

    return reviews, labels

reviews, labels = read_data_with_labels(df)

In [10]:

# Chuẩn bị dữ liệu đầu vào cho Word2Vec
input_gensim = [review.split() for review in reviews]

# Khởi tạo mô hình Word2Vec
model = Word2Vec(vector_size=128, window=5, min_count=1, workers=4, sg=1)

# Chuyển đổi vocab_list thành một từ điển với tần suất mặc định
word_freq = {word: 1 for word in vocab_list}

# Thiết lập danh sách từ vựng và vector của từng từ
model.build_vocab_from_freq(word_freq)

# Thiết lập các tham số cho mô hình
model.train(input_gensim, total_examples=len(input_gensim), epochs=10)

# Lưu mô hình
model.save("word.model")

In [11]:
# Load the Word2Vec model
model_embedding = Word2Vec.load('word.model')

max_seq = 200
embedding_size = 128

def comment_embedding(comment):
    matrix = np.zeros((max_seq, embedding_size))
    words = comment.split()
    lencmt = len(words)

    for i in range(max_seq):
        indexword = i % lencmt
        if (max_seq - i < lencmt):
            break
        if words[indexword] in model_embedding.wv:
            matrix[i] = model_embedding.wv[words[indexword]]
    matrix = np.array(matrix)
    return matrix

Mã hóa one-hot cho label

In [12]:
from sklearn.preprocessing import OneHotEncoder

# Mã hóa sentiment thành các vectơ theo yêu cầu
sentiment_mapping = {
    'trung tính': [1, 0, 0],
    'tích cực': [0, 1, 0],
    'tiêu cực': [0, 0, 1]
}

# Ánh xạ các nhãn sentiment sang vectơ mã hóa
df['Sentiment'] = df['Sentiment'].map(sentiment_mapping)

# Hiển thị kết quả
print(df['Sentiment'])

0       [0, 1, 0]
1       [0, 1, 0]
2       [0, 0, 1]
3       [0, 1, 0]
4       [1, 0, 0]
          ...    
1539    [0, 0, 1]
1540    [1, 0, 0]
1541    [0, 0, 1]
1542    [0, 1, 0]
1543    [0, 1, 0]
Name: Sentiment, Length: 1498, dtype: object


In [16]:
from tqdm import tqdm

# Assuming df['Comment'] contains your preprocessed comments
pre_reviews = df['Comment'].tolist()  # Convert DataFrame column to list if needed
labels = df['Sentiment'].tolist()  # Assuming you have a 'Sentiment' column with labels

train_data = []
label_data = []

for x in tqdm(pre_reviews):
    train_data.append(comment_embedding(x))
train_data = np.array(train_data)

for y in tqdm(labels):
    label_ = np.zeros(3)
    try:
        label_[int(y)] = 1
    except:
        label_[0] = 1
    label_data.append(label_)

100%|██████████| 1498/1498 [00:00<00:00, 21870.27it/s]


In [17]:
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf
from keras.preprocessing import sequence

sequence_length = 200
embedding_size = 128
num_classes = 3
filter_sizes = 3
num_filters = 150
epochs = 50
batch_size = 30
learning_rate = 0.01
dropout_rate = 0.5

In [19]:
# Define your model architecture
model = keras.Sequential()
model.add(layers.Conv2D(num_filters, (filter_sizes, embedding_size),
                        padding='valid',
                        input_shape=(sequence_length, embedding_size, 1), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(198, 1)))
model.add(layers.Dropout(dropout_rate))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

# Compile the model with the Adam optimizer
adam = tf.keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

print(model.summary())


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 198, 1, 150)    │        57,750 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 1, 1, 150)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1, 1, 150)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 150)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        19,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 77,465 (302.60 KB)

 Trainable params: 77,465 (302.60 KB)

 Non-trainable params: 0 (0.00 B)

None


In [28]:
# Assuming train_data and label_data are already prepared
x_train = np.array(train_data)  # Convert train_data to numpy array
y_train = np.array(label_data)  # Convert label_data to numpy array

# Now you can proceed with model training
model.fit(x=x_train[:1000], y=y_train[:1000], batch_size=batch_size, verbose=1, epochs=epochs, validation_data=(x_train[1000:1498], y_train[1000:1498]))

# Save the trained model
model.save('models.h5')

Epoch 1/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 6s 168ms/step - accuracy: 1.0000 - loss: 2.7056e-06 - val_accuracy: 1.0000 - val_loss: 6.9371e-07
Epoch 2/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 10s 148ms/step - accuracy: 1.0000 - loss: 3.0202e-06 - val_accuracy: 1.0000 - val_loss: 6.4991e-07
Epoch 3/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 152ms/step - accuracy: 1.0000 - loss: 3.4141e-06 - val_accuracy: 1.0000 - val_loss: 6.0658e-07
Epoch 4/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 139ms/step - accuracy: 1.0000 - loss: 3.6357e-06 - val_accuracy: 1.0000 - val_loss: 5.5296e-07
Epoch 5/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 139ms/step - accuracy: 1.0000 - loss: 3.1173e-06 - val_accuracy: 1.0000 - val_loss: 5.1586e-07
Epoch 6/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 4s 118ms/step - accuracy: 1.0000 - loss: 1.9549e-06 - val_accuracy: 1.0000 - val_loss: 4.8976e-07
Epoch 7/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 4s 106ms/step - accuracy: 1.0000 - loss: 2.4484e-06 - val_accuracy: 1.0000 - val_loss: 4.6032e-07
Epoch 8/50
34/34 ━━━━━━━━━━━━━━━━━━━━ 5s 135ms/

In [21]:
def pre_process(text):
    text = normalize_text(text)
    return text

In [26]:
text = "tệ"
text = pre_process(text)

maxtrix_embedding = np.expand_dims(comment_embedding(text), axis=0)
maxtrix_embedding = np.expand_dims(maxtrix_embedding, axis=3)

result = model.predict(maxtrix_embedding)
result = np.argmax(result)
print("Label predict: ", result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Label predict:  0
